# Cylindrical Vessel

The NEXT pressure vessels (PV), lead shield (PB) and copper shield (CU or CS) can be expressed as cylindrical vessels. 

In [1]:
import os
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

import time
import glob
import tables as tb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import numpy.testing as npt
from operator import itemgetter, attrgetter
%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.rcParams["figure.figsize"] = 10, 8
plt.rcParams["font.size"     ] = 14
pd.options.display.float_format = '{:.2g}'.format

2017-12-17 10:57:41


In [2]:
from math import sqrt, pi
from pynext.system_of_units import *
from pynext.Material import PhysicalMaterial
from pynext.Material import RadioactiveMaterial
from pynext.Material import PVMaterial
from pynext.Material import SelfAtt
from pynext.PhysicalVolume import PhysicalVolume
from pynext.Shapes import CylinderShell
from pynext.Shapes import Disk

In [3]:
from pynext.Material import vacuum, ti316, cu12, cu03, pb
from pynext.Material import GXe

In [4]:
from pynext.CylindricalVessel import CVD, CVA
from pynext.CylindricalVessel import CylindricalVessel

In [5]:
from pynext.NextData import NextPVData
from pynext.NextData import RFlux

In [6]:
from pynext.activity_functions import activity_lsc_gammas_through_CV
from pynext.activity_functions import activity_gammas_transmitted_CV
from pynext.activity_functions import activity_of_CV
from pynext.activity_functions import print_activity
from pynext.activity_functions import activity_table

In [7]:
from collections import namedtuple

## Cylindrical Vessel

A cylindrical vessel contains two physical volumes fille with the same material, one describing the body (e.g the cylinder itself) and the other describing the heads (the end-cups). 

## NEXT PV

### NEXT-100 PV dimensions

In [12]:
n100d = NextPVData()

In [13]:
n100d



        Next100PVData
        ------------------

        PV :
        inner diameter  = 1360.00 mm
        inner radius    =  680.00 mm
        outer diameter  = 1370.00 mm
        outer radius    =  685.00 mm
        body thickness  =   10.00 mm
        head thickness  =   12.00 mm
        length          = 1600.00 mm

        CS :
        inner diameter  = 1240.00 mm
        inner radius    =  620.00 mm
        outer diameter  = 1360.00 mm
        outer radius    =  680.00 mm
        body thickness  =  120.00 mm
        head thickness  =  120.00 mm
        length          = 1600.00 mm

        PB :
        inner diameter  = 1370.00 mm
        inner radius    =  685.00 mm
        outer diameter  = 1570.00 mm
        outer radius    =  785.00 mm
        body thickness  =  200.00 mm
        head thickness  =  200.00 mm
        length          = 1600.00 mm

    

In [14]:
cvd_pv = CVD(name    = 'Next100PV',
                 R       = n100d.pv_inner_radius,
                 th_body = n100d.pv_body_thickness,
                 L       = n100d.pv_length,
                 th_head = n100d.pv_head_thickness)

n100_pv = CylindricalVessel(name=cvd_pv.name, material=ti316, cvd=cvd_pv)

In [15]:
n100_pv


        Cylindrical Vessel:

        ----------------
        name      = Next100PV
        material  = 316ti

        specific activity of material:
        Bi-214    =    1.00 mBq/kg
        Tl-208    =    0.15 mBq/kg

        body:
        R         =  680.00 mm
        thickness =   10.00 mm
        length    = 1600.00 mm
        surface   = 6.84e+06 mm2
        volume    = 6.89e+07 mm3
        mass      =  541.96 kg
        activity Bi-214 =  541.96 mBq, self-shielded = 195.19 mBq
        activity Tl-208 =   81.29 mBq, self-shielded =  29.28 mBq
        transmittance   = 7.36e-01

        heads:
        thickness =   12.00 mm
        surface   = 2.91e+06 mm2
        volume    = 3.49e+07 mm3
        mass      =  274.38 kg
        activity Bi-214 =  137.19 mBq, self-shielded =  47.44 mBq
        activity Tl-208 =   20.58 mBq, self-shielded =   7.12 mBq
        transmittance   = 6.92e-01

        

## NEXT envelop

An envelop of NEXT dimensions filled with vacuum that can be used to compute gamma fluxes

In [16]:
n100_envelop = CylindricalVessel(name='Next100Envelop', material=vacuum, cvd=cvd_pv)
n100_envelop


        Cylindrical Vessel:

        ----------------
        name      = Next100PV
        material  = vacuum

        specific activity of material:
        Bi-214    =    0.00 mBq/kg
        Tl-208    =    0.00 mBq/kg

        body:
        R         =  680.00 mm
        thickness =   10.00 mm
        length    = 1600.00 mm
        surface   = 6.84e+06 mm2
        volume    = 6.89e+07 mm3
        mass      =    0.00 kg
        activity Bi-214 =    0.00 mBq, self-shielded =   0.00 mBq
        activity Tl-208 =    0.00 mBq, self-shielded =   0.00 mBq
        transmittance   = 1.00e+00

        heads:
        thickness =   12.00 mm
        surface   = 2.91e+06 mm2
        volume    = 3.49e+07 mm3
        mass      =    0.00 kg
        activity Bi-214 =    0.00 mBq, self-shielded =   0.00 mBq
        activity Tl-208 =    0.00 mBq, self-shielded =   0.00 mBq
        transmittance   = 1.00e+00

        

##  Lead Castle

In [17]:
cvd_pb = CVD(name    ='PBShield', 
             R       =  n100d.pb_inner_diameter, 
             th_body = n100d.pb_body_thickness, 
             L       = n100d.pb_length, 
             th_head = n100d.pb_head_thickness)

In [18]:
n100_pb = CylindricalVessel(name='Next100Pb', material=pb, cvd=cvd_pb)
n100_pb


        Cylindrical Vessel:

        ----------------
        name      = PBShield
        material  = Pb

        specific activity of material:
        Bi-214    =    0.37 mBq/kg
        Tl-208    =    0.07 mBq/kg

        body:
        R         = 1370.00 mm
        thickness =  200.00 mm
        length    = 1600.00 mm
        surface   = 1.38e+07 mm2
        volume    = 2.96e+09 mm3
        mass      = 33487.07 kg
        activity Bi-214 = 12390.21 mBq, self-shielded = 368.65 mBq
        activity Tl-208 = 2444.56 mBq, self-shielded =  72.73 mBq
        transmittance   = 4.68e-05

        heads:
        thickness =  200.00 mm
        surface   = 1.18e+07 mm2
        volume    = 2.36e+09 mm3
        mass      = 26722.74 kg
        activity Bi-214 = 4943.71 mBq, self-shielded = 157.83 mBq
        activity Tl-208 =  975.38 mBq, self-shielded =  31.14 mBq
        transmittance   = 4.68e-05

        

##  ICS (aka, CS, CU)

In [19]:
cvd_cu = CVD(name    ='CUShield', 
             R       = n100d.cs_inner_diameter, 
             th_body = n100d.cs_body_thickness, 
             L       = n100d.cs_length, 
             th_head = n100d.cs_head_thickness)
cvd_cu

CVD(name='CUShield', R=1240.0, th_body=120.0, L=1600.0, th_head=120.0)

In [20]:
n100_cu = CylindricalVessel(name='Next100CU', material=cu12, cvd=cvd_cu)
n100_cu


        Cylindrical Vessel:

        ----------------
        name      = CUShield
        material  = CuUpperLimits

        specific activity of material:
        Bi-214    =    0.01 mBq/kg
        Tl-208    =    0.00 mBq/kg

        body:
        R         = 1240.00 mm
        thickness =  120.00 mm
        length    = 1600.00 mm
        surface   = 1.25e+07 mm2
        volume    = 1.57e+09 mm3
        mass      = 14051.82 kg
        activity Bi-214 =  168.62 mBq, self-shielded =  12.12 mBq
        activity Tl-208 =   19.67 mBq, self-shielded =   1.41 mBq
        transmittance   = 1.51e-02

        heads:
        thickness =  120.00 mm
        surface   = 9.66e+06 mm2
        volume    = 1.16e+09 mm3
        mass      = 10387.53 kg
        activity Bi-214 =   62.33 mBq, self-shielded =   4.70 mBq
        activity Tl-208 =    7.27 mBq, self-shielded =   0.55 mBq
        transmittance   = 1.51e-02

        